# Mortality Dataset

In [1]:
import pandas as pd
import numpy as np
import datetime

import seaborn as sns
import matplotlib.pyplot as plt

In [9]:
data = pd.read_csv("mort.csv")

# Output some basic information about the data
print "Shape: {}".format(data.shape)
print "___________________________________"
print "Columns: \n{}".format(list(data.columns.values))
print "___________________________________"
print data['Category'].value_counts()
print "___________________________________"
data.head()

Shape: (67074, 30)
___________________________________
Columns: 
['Location', 'FIPS', 'Category', 'Mortality Rate, 1980*', 'Mortality Rate, 1980* (Min)', 'Mortality Rate, 1980* (Max)', 'Mortality Rate, 1985*', 'Mortality Rate, 1985* (Min)', 'Mortality Rate, 1985* (Max)', 'Mortality Rate, 1990*', 'Mortality Rate, 1990* (Min)', 'Mortality Rate, 1990* (Max)', 'Mortality Rate, 1995*', 'Mortality Rate, 1995* (Min)', 'Mortality Rate, 1995* (Max)', 'Mortality Rate, 2000*', 'Mortality Rate, 2000* (Min)', 'Mortality Rate, 2000* (Max)', 'Mortality Rate, 2005*', 'Mortality Rate, 2005* (Min)', 'Mortality Rate, 2005* (Max)', 'Mortality Rate, 2010*', 'Mortality Rate, 2010* (Min)', 'Mortality Rate, 2010* (Max)', 'Mortality Rate, 2014*', 'Mortality Rate, 2014* (Min)', 'Mortality Rate, 2014* (Max)', '% Change in Mortality Rate, 1980-2014', '% Change in Mortality Rate, 1980-2014 (Min)', '% Change in Mortality Rate, 1980-2014 (Max)']
___________________________________
Diabetes, urogenital, blood, and en

,Location,FIPS,Category,"Mortality Rate, 1980*","Mortality Rate, 1980* (Min)","Mortality Rate, 1980* (Max)","Mortality Rate, 1985*","Mortality Rate, 1985* (Min)","Mortality Rate, 1985* (Max)","Mortality Rate, 1990*",...,"Mortality Rate, 2005* (Max)","Mortality Rate, 2010*","Mortality Rate, 2010* (Min)","Mortality Rate, 2010* (Max)","Mortality Rate, 2014*","Mortality Rate, 2014* (Min)","Mortality Rate, 2014* (Max)","% Change in Mortality Rate, 1980-2014","% Change in Mortality Rate, 1980-2014 (Min)","% Change in Mortality Rate, 1980-2014 (Max)"
0,United States,NaN,Neonatal disorders,9.18,8.83,9.93,6.91,6.73,7.36,6.09,...,4.55,3.75,3.43,3.85,3.32,3.02,3.45,-63.85,-68.95,-61.55
1,Alabama,1.0,Neonatal disorders,11.03,10.57,12.00,8.51,8.25,9.12,7.52,...,6.52,5.58,5.16,5.78,5.10,4.67,5.32,-53.82,-60.08,-50.77
2,"Autauga County, Alabama",1001.0,Neonatal disorders,9.58,8.37,11.02,7.50,6.56,8.58,6.76,...,6.30,4.83,4.19,5.49,4.56,3.96,5.21,-52.43,-60.01,-46.16
3,"Baldwin County, Alabama",1003.0,Neonatal disorders,8.75,7.86,9.81,6.54,5.88,7.27,5.76,...,5.03,4.02,3.53,4.48,3.68,3.25,4.08,-57.97,-64.96,-52.87
4,"Barbour County, Alabama",1005.0,Neonatal disorders,12.30,10.64,14.15,9.26,8.13,10.60,8.12,...,7.89,5.99,5.16,6.84,5.43,4.63,6.24,-55.86,-62.49,-50.32
